### **Code : (2) 비용 관리 표준화 패턴 모델링**

- Project : 2024 데이터바우처 지원사업
- Writer : Donghyeon Kim
- Update : 2024.10.05.

#### **0. 라이브러리 및 초기 경로 설정**

In [1]:
from pathlib import Path
import os
import pandas as pd
import numpy as np
import xlrd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import random
import tensorflow as tf
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import PatternFill
from matplotlib import font_manager, rc, rcParams, ticker
from sklearn.metrics import mean_absolute_percentage_error
from autots import AutoTS
from datetime import datetime, date
from statsmodels.api import OLS
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from keras.layers import LSTM, Dense, Input
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm

In [2]:
dr = 'C:/'
folder_1 = 'Users/USER/Dropbox/8. 회사업무/1. 산학협력프로젝트/2024년/9. [선정] 2024 데이터바우처 지원사업/7. 분석'
root = dr + folder_1

folder_2 = '1_rawdata'
folder_path = root + '/' + folder_2

os.chdir(folder_path)

In [3]:
# 회사이름
company_name = os.listdir()[6] # Available Index : 0 ~ 9 (Total : 10)

# 회사이름 포함 경로
final_path = folder_path + '/' + company_name
os.chdir(final_path)

---

#### **1. 매입(세금계산서) Data Load**

In [5]:
# '매입(세금계산서)' 파일명 검색
file_name = [file for file in os.listdir() if '매입(세금계산서)' in file]  # 매입(세금계산서) 파일 1개

# 파일 존재 여부 확인
if len(file_name) > 0:
    # Data Frame
    df = pd.read_excel(file_name[0], skiprows=1)
    # Data Frame Head(2023)
    df_without_account = df.drop(columns=['공급자상호', '내용'])
    print(df_without_account[df_without_account['작성일자'].str.contains('2023-12')].head())
else:
    # 파일 없음 오류 문구
    print("매입(세금계산서) 파일이 없습니다.")

          작성일자  증빙    합계금액     용도  프로젝트/현장                            비고   메모
61  2023-12-31  세계   59400  지급수수료      NaN   결의서_202401_5, 2024-01-10 계좌  NaN
62  2023-12-28  세계  993300    자재비      NaN  결의서_202312_19, 2023-12-29 계좌  NaN
63  2023-12-27  세계  880000    임차료      NaN  결의서_202312_18, 2023-12-29 계좌  NaN
64  2023-12-26  세계  286000   소모품비      NaN  결의서_202312_17, 2023-12-29 계좌  NaN
65  2023-12-18   계   17020  건물관리비      NaN  결의서_202312_16, 2023-12-29 계좌  NaN


In [6]:
# 회사명에 따른 결과물 저장 경로
result_root = os.path.join(root, 'Model(2nd)')
company_result_root = os.path.join(result_root, company_name, '1_매입(세금계산서)')
if not os.path.isdir(company_result_root):
    os.makedirs(company_result_root)

#### **1) 매입(세금계산서) : 비율 수치 & Graph**

In [13]:
# 1) 용도별 비율 수치 계산
print('용도별 비율 계산 중')
usage_summary = df.groupby('용도')['합계금액'].sum().reset_index()  # '사용금액' 대신 '합계금액' 사용
usage_summary['비율'] = round((usage_summary['합계금액'] / usage_summary['합계금액'].sum()) * 100, 3)

# '합계금액' 열에 콤마를 추가한 형식으로 변환
usage_summary['합계금액'] = usage_summary['합계금액'].apply(lambda x: f'{int(x):,}')

print('----------------------------------------------')
print(usage_summary)
print('----------------------------------------------')

# 결과물 최종 경로
output_file = os.path.join(company_result_root, '결과1_용도 비율 표.xlsx')

# 엑셀 파일로 저장
usage_summary.to_excel(output_file, index=False)
print('용도별 비율 표 Excel 저장 완료')

용도별 비율 계산 중
----------------------------------------------
      용도         합계금액      비율
0  건물관리비    8,992,900   3.285
1    광고비       22,000   0.008
2  도서인쇄비      738,100   0.270
3     매입  115,054,919  42.032
4   소모품비   12,418,500   4.537
5    임차료   29,161,000  10.653
6    자재비   82,765,100  30.236
7  지급수수료   22,684,639   8.287
8    통신비    1,894,474   0.692
----------------------------------------------
용도별 비율 표 Excel 저장 완료


In [14]:
# 2) 3% 이상인 항목만 그래프에 표시 (표에는 모두 포함)
filtered_data = usage_summary[usage_summary['비율'] >= 3]

# Seaborn 스타일 설정 및 글씨 설정
sns.set_theme(style='whitegrid')
plt.figure(figsize=(10, 6), dpi=300)
plt.rcParams['font.family'] = 'NanumGothicOTF' # 'NanumGothic' or 'NanumGothicOTF'

# 컬러맵 설정
cmap = plt.get_cmap('tab20')
colors = [cmap(i / len(filtered_data)) for i in range(len(filtered_data))]

# Seaborn을 사용한 barplot 생성
bar_plot = sns.barplot(x='비율', y='용도', data=filtered_data,
                       hue='용도', palette='tab20', dodge=False, legend=False)

# 제목 및 레이블 설정에 폰트 적용
bar_plot.set_title('용도별 사용금액 비율 (3% 이상만 표시)', fontsize=16, fontweight='bold')
bar_plot.set_xlabel('비율 (%)', fontsize=14, fontweight='bold')
bar_plot.set_ylabel('용도', fontsize=14, fontweight='bold')

# 막대마다 비율(%) 수치 표시
for index, value in enumerate(filtered_data['비율']):
    plt.text(value + 0.5, index, f'{value:.1f}%', va='center', fontsize=12, color='black')  # 수치를 막대 옆에 표시

# 그래프 저장
graph_path = os.path.join(company_result_root, '결과1_용도 비율 그래프.png')
plt.tight_layout()
plt.savefig(graph_path)
plt.close()

print('용도별 비율 그래프 저장 완료')

용도별 비율 그래프 저장 완료


#### **2) 매입(세금계산서) : 월별 추세 수치 & Graph**

In [15]:
# 분석 2 : 용도/월(Month)별 금액 추세 수치 + Graph
print('용도/월별 금액 추세 계산 중')

# Seaborn 스타일 설정 및 글씨 설정
sns.set_theme(style='whitegrid')
plt.figure(figsize=(20, 10), dpi=300)
plt.rcParams['font.family'] = 'NanumGothicOTF'

# 작성일자에서 '월' 변수를 추출하여 추가
df['작성일자'] = pd.to_datetime(df['작성일자'])  # 작성일자를 datetime 형식으로 변환
df['월'] = df['작성일자'].dt.to_period('M')  # '월' 변수 생성 (YYYY-MM 형태)

# 2023년까지의 데이터만 필터링
df_filtered = df[df['작성일자'] <= '2023-12-31']  # 2023년까지 데이터 선택

# 금액 추세 수치 계산
monthly_trend = df_filtered.groupby(['월', '용도'])['합계금액'].sum().reset_index()
trend_pivot = monthly_trend.pivot(index='월', columns='용도', values='합계금액').fillna(0)
trend_pivot.index = trend_pivot.index.to_timestamp().strftime('%Y-%m')  # Timestamp → 'YYYY-MM' 형태 변환

# 각 용도별 총 사용 금액 계산
total_usage_by_type = trend_pivot.sum(axis=0)

# 총 사용 금액이 가장 많은 상위 5개의 용도 선택
top_5_usage_types = total_usage_by_type.nlargest(5).index

# 상위 5개의 용도만 필터링
filtered_trend_pivot = trend_pivot[top_5_usage_types]

# Seaborn 라인 차트 생성
for usage_type in filtered_trend_pivot.columns:
    sns.lineplot(x=filtered_trend_pivot.index, y=filtered_trend_pivot[usage_type], marker='o', label=usage_type)

# 제목 및 레이블 설정
plt.title('월별 용도별 사용금액 추세 (Top 5)', fontsize=16, fontweight='bold')
plt.xlabel('월', fontsize=14, fontweight='bold')
plt.ylabel('사용금액', fontsize=14, fontweight='bold')

# y축 금액 포맷을 일반 숫자 형식으로 변경 (콤마 포함)
ax = plt.gca()  # 현재 축 가져오기

# FuncFormatter를 사용하여 금액을 콤마로 구분된 형식으로 변경
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{int(x):,}'))

# x축 레이블 간격을 조정하고 레이블을 45도 회전
plt.xticks(rotation=45, ha='right')  # ha='right'는 레이블을 오른쪽에 정렬

# Legend를 그래프 밖으로 위치 조정
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=14)

# 그리드 추가
plt.grid(True)

# 그래프 저장
line_chart_path = os.path.join(company_result_root, '결과2_용도 및 월별 추세.png')
plt.tight_layout()
plt.savefig(line_chart_path, bbox_inches='tight')
plt.close()

print('용도 및 월별 금액 추세 그래프 (2023년까지) 저장 완료')

용도/월별 금액 추세 계산 중
용도 및 월별 금액 추세 그래프 (2023년까지) 저장 완료


#### **3) 매입(세금계산서) : 비용 적정성 결과**

In [16]:
# 분석 3 : 용도별 전월 대비 비용 적정성 분석 결과
print('전월 대비 비용 적정성 계산 중')

# 금액 추세 수치 계산
monthly_trend = df_filtered.groupby(['월', '용도'])['합계금액'].sum().reset_index()
trend_pivot = monthly_trend.pivot(index='월', columns='용도', values='합계금액').fillna(0)
trend_pivot.index = trend_pivot.index.to_timestamp().strftime('%Y-%m')  # Timestamp → 'YYYY-MM' 형태 변환

# 전월 대비 이번 달의 적정성 평가
comparison_results = pd.DataFrame()

# 첫 달은 비교 없이 그대로 금액을 표시
first_month_result = pd.DataFrame(trend_pivot.iloc[0].apply(lambda x: f'{int(x):,}')).T
first_month_result.index = [trend_pivot.index[0]]
comparison_results = pd.concat([comparison_results, first_month_result], ignore_index=False)

for k in range(1, len(trend_pivot)):  # 첫 달 이후 달부터 평가
    last_month = trend_pivot.iloc[k - 1]  # 전월 데이터
    current_month = trend_pivot.iloc[k]  # 이번 달 데이터
    comparison = current_month - last_month  # 전월과의 차이 계산
    
    # 적정성 평가
    comparison_eval = comparison.apply(
        lambda x: '낮음' if x < 0 else ('높음' if x > 0 else '보통')  # 전월 대비 적정성 평가
    )
    
    # '비용 적정성(변동액)' 형태로 정리
    result_with_values = comparison_eval + '(' + comparison.fillna(0).round(0).astype(int).apply(lambda x: f'{x:,}') + ')'
    
    # 결과 정리
    monthly_result = pd.DataFrame(result_with_values).T
    monthly_result.index = [trend_pivot.index[k]]
    comparison_results = pd.concat([comparison_results, monthly_result], ignore_index=False)

# 엑셀 저장 경로
output_file = os.path.join(company_result_root, '결과3_전월 대비 비용 적정성 결과.xlsx')

# Excel 파일 생성
wb = Workbook()
ws = wb.active
ws.title = "전월 대비 비용 적정성 분석 결과"

# 첫 번째 행에 열 제목을 입력
ws.append(['날짜'] + list(comparison_results.columns))

# 데이터프레임을 엑셀 시트로 전송
for r_idx, row in enumerate(dataframe_to_rows(comparison_results, index=True, header=True), 1):
    for c_idx, value in enumerate(row, 1):
        ws.cell(row=r_idx, column=c_idx, value=value)
        
        # 첫 번째 달(첫 행)에는 색상을 부여하지 않음
        if r_idx > 1 and c_idx > 1 and isinstance(value, str):  # 첫 번째 행과 열은 제외하고 색상 설정
            if '(' in value:  # 괄호가 있는지 확인
                eval_text = value.split('(')[0]  # "낮음", "보통", "높음" 추출
                number_value = value.split('(')[1].replace(')', '')  # 금액 부분만 추출
                
                if eval_text == '낮음':
                    ws.cell(row=r_idx, column=c_idx, value=number_value).fill = PatternFill(start_color="32CD32", end_color="32CD32", fill_type="solid")  # Limegreen
                elif eval_text == '보통':
                    ws.cell(row=r_idx, column=c_idx, value=number_value).fill = PatternFill(start_color="FFD700", end_color="FFD700", fill_type="solid")  # Gold
                elif eval_text == '높음':
                    ws.cell(row=r_idx, column=c_idx, value=number_value).fill = PatternFill(start_color="FF6347", end_color="FF6347", fill_type="solid")  # Tomato

# 엑셀 파일 저장
wb.save(output_file)
print("전월 대비 비용 적정성 결과 저장 완료")

전월 대비 비용 적정성 계산 중
전월 대비 비용 적정성 결과 저장 완료


#### **4) 매입(세금계산서) : 향후 추세 분석/예측 결과**

In [17]:
# MAPE 계산 함수
def calculate_mape(true_values, predicted_values):
    mape = mean_absolute_percentage_error(true_values, predicted_values) * 100
    return round(mape, 2)  # 소수점 아래 2자리에서 반올림

# LSTM Model (Random Seed)
def lstm_forecast(train_data, steps=12):
    # # 시드 고정 (LSTM에만 적용)
    # np.random.seed(seed)
    # random.seed(seed)
    # tf.random.set_seed(seed)

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_train_data = scaler.fit_transform(train_data.values.reshape(-1, 1))

    model = Sequential()
    model.add(Input(shape=(1, 1)))  # Input 객체로 첫 번째 레이어 정의
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    X_train = np.array([scaled_train_data[i] for i in range(len(scaled_train_data) - 1)])
    y_train = np.array([scaled_train_data[i+1] for i in range(len(scaled_train_data) - 1)])

    model.fit(X_train, y_train, epochs=100, verbose=0)

    last_value = scaled_train_data[-1]
    predictions = []
    for _ in range(steps):
        pred = model.predict(last_value.reshape(1, 1), verbose=0)
        predictions.append(pred[0][0])
        last_value = np.array([[pred[0][0]]])

    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
    return pd.Series(predictions.flatten())

In [20]:
# 경고 무시 설정
warnings.filterwarnings("ignore", category=FutureWarning)

# Seaborn 스타일 설정 및 글씨 설정
sns.set_theme(style='whitegrid')
plt.figure(figsize=(20, 10), dpi=300)
plt.rcParams['font.family'] = 'NanumGothicOTF'

# 분석 4 : 미래 12개월 금액 추세 예측 결과(LSTM + MAPE 평가)
print('향후 12개월 추세 예측 및 위험도 평가 실행 중')
forecast_results = {} # 추세 예측 결과
risk_eval_results = {} # 위험도 평가 결과
mape_results = {}  # MAPE 평가 결과
forecast_plots = {} # 용도별 예측 그래프 경로

for col in tqdm(trend_pivot.columns, desc='Forecasting'):
    if len(trend_pivot[col].dropna()) < 3:  # 예측 시도 전에 데이터 길이 체크
        print(f'예측 불가 : {col} → 데이터가 3개 미만')
        forecast_results[col] = pd.Series([None] * 12)  # 예측 불가
        risk_eval_results[col] = [None] * 12  # 예측 불가
        mape_results[col] = None  # MAPE 계산 불가
        continue
    
    # 0 값 비율 계산
    zero_ratio = (trend_pivot[col] == 0).mean()
    
    # 0 값 비율이 10%를 넘으면 예측을 건너뜀
    if zero_ratio > 0.1:  # 10% 이상 0이면 예측 건너뛰기
        print(f'{col}: 예측 건너뜀 (0 값 비율 = {zero_ratio:.2%})')
        forecast_results[col] = pd.Series([None] * 12)  # 예측 불가
        risk_eval_results[col] = [None] * 12  # 예측 불가
        mape_results[col] = None  # MAPE 계산 불가
        continue

    try:
        # LSTM 모델 적용
        print('----------------------------------------------')
        print(f'{col}: LSTM Model 사용')
        forecast = lstm_forecast(trend_pivot[col].dropna(), steps=12)
        
        # 예측 결과 저장
        forecast_results[col] = forecast
        
        # MAPE 계산 (마지막 12개월의 실제 값과 비교)
        actual_values = trend_pivot[col].dropna()[-12:]
        if len(actual_values) == 12:
            mape = calculate_mape(actual_values, forecast)
            mape_results[col] = mape
            print(f'{col} MAPE: {mape:.2f}%')
        else:
            print(f'{col} MAPE 계산 불가: 데이터가 충분하지 않음')
            mape_results[col] = None
        
        # 인덱스를 명시적으로 날짜로 변환 (tz 오류 해결)
        trend_pivot.index = pd.to_datetime(trend_pivot.index, format='%Y-%m')
        
        # Seaborn lineplot을 사용하여 실제값과 예측값 그리기
        plt.figure(figsize=(20, 10), dpi=300)  # 모든 그래프의 크기를 동일하게 설정
        sns.lineplot(x=trend_pivot.index, y=trend_pivot[col], label='실제값', marker='o', color='blue')
        sns.lineplot(x=pd.date_range(start=trend_pivot.index[-1], periods=12, freq='M'), y=forecast, label='예측값 (LSTM Model)', linestyle='--', marker='x', color='red')

        # 제목 및 레이블 설정
        plt.title(f'{col} 향후 12개월 예상 금액', fontsize=16, fontweight='bold')
        plt.xlabel('날짜', fontsize=14, fontweight='bold')
        plt.ylabel('금액', fontsize=14, fontweight='bold')

        # y축 금액 포맷을 일반 숫자 형식으로 변경 (콤마 포함)
        ax = plt.gca()  # 현재 축 가져오기

        # FuncFormatter를 사용하여 금액을 콤마로 구분된 형식으로 변경
        ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{int(x):,}'))
        
        # y축의 값을 0부터 시작하도록 설정
        plt.ylim(bottom=0)

        # x축 레이블 간격을 조정하고 레이블을 45도 회전
        plt.xticks(rotation=45, ha='right')  # ha='right'는 레이블을 오른쪽에 정렬

        # Legend를 그래프 밖으로 위치 조정
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=14)

        # 그래프 저장
        forecast_plot_path = os.path.join(company_result_root, f'결과4_용도 {col} 예측 그래프.png')
        plt.tight_layout()
        plt.savefig(forecast_plot_path)
        plt.close()
        
        # 그래프 경로 저장
        forecast_plots[col] = forecast_plot_path
        
    except Exception as e:
        # 예측 실패 시 None 값 처리
        print(f'예측 오류 {col}: {e}')
        forecast_results[col] = pd.Series([None] * 12)
        risk_eval_results[col] = [None] * 12
        mape_results[col] = None
    
    print('----------------------------------------------')

# 예측 결과 + 위험도 평가 + MAPE 결과 정리
forecast_df = pd.DataFrame(forecast_results)
start_date = df['작성일자'].max()
forecast_df.index = pd.date_range(start=start_date, periods=12, freq='M')
forecast_df.index = forecast_df.index.strftime('%Y-%m')  # Index 'yyyy-mm' 형태 변환

# MAPE 결과 추가
mape_df = pd.DataFrame(mape_results, index=[0])
mape_df.index = ['MAPE']

# 예측 결과 + MAPE 평가 취합 및 정렬
forecast_combined_df = pd.concat([forecast_df, mape_df], axis=0)

# 엑셀 저장 경로 설정
output_file = os.path.join(company_result_root, '결과4_추세 예측 결과.xlsx')

# Excel 파일 생성 및 저장
wb = Workbook()
ws = wb.active
ws.title = "예측 결과"

# 첫 번째 행에 열 제목을 입력
ws.append(['날짜'] + list(forecast_combined_df.columns))

# 데이터프레임을 엑셀 시트로 전송 및 색상 적용
for r_idx, row in enumerate(forecast_combined_df.itertuples(), 1):
    for c_idx, value in enumerate(row, 0):  # 0에서 시작하여 열을 일치시킴
        if c_idx == 0:
            # 첫 번째 열에 날짜를 입력
            ws.cell(row=r_idx + 1, column=1, value=forecast_combined_df.index[r_idx - 1])
        else:
            # 나머지 열에 값 입력
            ws.cell(row=r_idx + 1, column=c_idx + 1, value=value)

            # MAPE 행에는 색상을 적용하지 않음
            if forecast_combined_df.index[r_idx - 1] != 'MAPE' and pd.notna(value):
                # 과거 평균값과 비교하여 색상 적용
                col_name = forecast_combined_df.columns[c_idx - 1]  # c_idx-1로 수정하여 열을 올바르게 참조
                
                # 예측하기 직전 달까지의 평균값 계산 (MAPE와 현재 행을 제외한 값들로 평균 계산)
                avg_past = trend_pivot[col_name].mean()

                # 값이 평균보다 작으면 초록색, 크면 빨간색, 비슷하면 주황색
                if value < avg_past:
                    ws.cell(row=r_idx + 1, column=c_idx + 1).fill = PatternFill(start_color="32CD32", end_color="32CD32", fill_type="solid")  # Limegreen
                elif value > avg_past:
                    ws.cell(row=r_idx + 1, column=c_idx + 1).fill = PatternFill(start_color="FF6347", end_color="FF6347", fill_type="solid")  # Tomato
                else:
                    ws.cell(row=r_idx + 1, column=c_idx + 1).fill = PatternFill(start_color="FFD700", end_color="FFD700", fill_type="solid")  # Gold

# 엑셀 파일 저장
wb.save(output_file)
print("예측 결과 및 위험도 분석 엑셀 저장 완료")

향후 12개월 추세 예측 및 위험도 평가 실행 중


Forecasting:   0%|          | 0/9 [00:00<?, ?it/s]

----------------------------------------------
건물관리비: LSTM Model 사용
건물관리비 MAPE: 6.71%


Forecasting:  11%|█         | 1/9 [00:08<01:10,  8.86s/it]

----------------------------------------------
광고비: 예측 건너뜀 (0 값 비율 = 97.30%)
도서인쇄비: 예측 건너뜀 (0 값 비율 = 81.08%)
매입: 예측 건너뜀 (0 값 비율 = 62.16%)
소모품비: 예측 건너뜀 (0 값 비율 = 24.32%)
----------------------------------------------
임차료: LSTM Model 사용
임차료 MAPE: 21.32%


...

Forecasting:  89%|████████▉ | 8/9 [00:35<00:04,  4.34s/it]

----------------------------------------------
----------------------------------------------
통신비: LSTM Model 사용
통신비 MAPE: 27.52%


Forecasting: 100%|██████████| 9/9 [00:43<00:00,  4.88s/it]

----------------------------------------------
예측 결과 및 위험도 분석 엑셀 저장 완료


<Figure size 6000x3000 with 0 Axes>

---

#### **2. 매입(카드/페이) Data Load**

In [27]:
# '매입(카드_페이)' 파일명 검색
file_name = [file for file in os.listdir() if '매입(카드_페이)' in file]  # 매입(카드/페이) 파일 1개

# 파일 존재 여부 확인
if len(file_name) > 0:
    # Data Frame
    df = pd.read_excel(file_name[0], skiprows=1)
    # Data Frame Head(2023)
    df_without_account = df.drop(columns=['사용처', '카드번호', '사용처업종'])
    print(df_without_account[df_without_account['사용일자'].str.contains('2023-12')].head())
else:
    # 파일 없음 오류 문구
    print("매입(카드/페이) 파일이 없습니다.")

           사용일자  증빙   사용금액 사용처과세유형     용도       내용 공제/불공제  비고  메모
822  2023-12-30 NaN   3540      일반  복리후생비      간식대      Y NaN NaN
823  2023-12-30 NaN   1500      일반  복리후생비      음료대      Y NaN NaN
824  2023-12-29 NaN   4050      일반  복리후생비      간식대      Y NaN NaN
825  2023-12-29 NaN  14700      일반   소모품비  소모품 구입대      Y NaN NaN
826  2023-12-29 NaN   3500      일반  복리후생비      간식대      Y NaN NaN


In [28]:
# 회사명에 따른 결과물 저장 경로
result_root = os.path.join(root, 'Model(2nd)')
company_result_root = os.path.join(result_root, company_name, '2_매입(카드_페이)')
if not os.path.isdir(company_result_root):
    os.makedirs(company_result_root)

#### **1) 매입(카드/페이) : 비율 수치 & Graph**

In [29]:
# 1) 용도별 비율 수치 계산
print('용도별 비율 계산 중')
usage_summary = df.groupby('용도')['사용금액'].sum().reset_index()  # '사용금액'
usage_summary['비율'] = round((usage_summary['사용금액'] / usage_summary['사용금액'].sum()) * 100, 3)

# '사용금액' 열에 콤마를 추가한 형식으로 변환
usage_summary['사용금액'] = usage_summary['사용금액'].apply(lambda x: f'{int(x):,}')

print('----------------------------------------------')
print(usage_summary)
print('----------------------------------------------')

# 결과물 최종 경로
output_file = os.path.join(company_result_root, '결과1_용도 비율 표.xlsx')

# 엑셀 파일로 저장
usage_summary.to_excel(output_file, index=False)
print('용도별 비율 표 Excel 저장 완료')

용도별 비율 계산 중
----------------------------------------------
         용도        사용금액      비율
0   가지급금 출금   1,500,000   1.139
1       교통비       6,400   0.005
2      기타비용   6,544,510   4.968
3     도서인쇄비     970,500   0.737
4       보험료   7,874,890   5.978
5     복리후생비  48,644,980  36.931
6      소모품비  39,531,970  30.012
7     여비교통비     141,900   0.108
8       운반비     115,200   0.087
9       자재비     440,700   0.335
10      접대비   1,050,500   0.798
11      주차비      30,100   0.023
12    지급수수료     767,810   0.583
13    차량수리비      51,000   0.039
14    차량유지비  23,805,409  18.073
15      출장비      94,500   0.072
16      통신비      77,870   0.059
17      회식비      72,000   0.055
----------------------------------------------
용도별 비율 표 Excel 저장 완료


In [30]:
# 2) 3% 이상인 항목만 그래프에 표시 (표에는 모두 포함)
filtered_data = usage_summary[usage_summary['비율'] >= 3]

# Seaborn 스타일 설정 및 글씨 설정
sns.set_theme(style='whitegrid')
plt.figure(figsize=(10, 6), dpi=300)
plt.rcParams['font.family'] = 'NanumGothicOTF' # 'NanumGothic' or 'NanumGothicOTF'

# 컬러맵 설정
cmap = plt.get_cmap('tab20')
colors = [cmap(i / len(filtered_data)) for i in range(len(filtered_data))]

# Seaborn을 사용한 barplot 생성
bar_plot = sns.barplot(x='비율', y='용도', data=filtered_data,
                       hue='용도', palette='tab20', dodge=False, legend=False)

# 제목 및 레이블 설정에 폰트 적용
bar_plot.set_title('용도별 사용금액 비율 (3% 이상만 표시)', fontsize=16, fontweight='bold')
bar_plot.set_xlabel('비율 (%)', fontsize=14, fontweight='bold')
bar_plot.set_ylabel('용도', fontsize=14, fontweight='bold')

# 막대마다 비율(%) 수치 표시
for index, value in enumerate(filtered_data['비율']):
    plt.text(value + 0.5, index, f'{value:.1f}%', va='center', fontsize=12, color='black')  # 수치를 막대 옆에 표시

# 그래프 저장
graph_path = os.path.join(company_result_root, '결과1_용도 비율 그래프.png')
plt.tight_layout()
plt.savefig(graph_path)
plt.close()

print('용도별 비율 그래프 저장 완료')

용도별 비율 그래프 저장 완료


#### **2) 매입(카드/페이) : 월별 추세 수치 & Graph**

In [31]:
# 분석 2 : 용도/월(Month)별 금액 추세 수치 + Graph
print('용도/월별 금액 추세 계산 중')

# Seaborn 스타일 설정 및 글씨 설정
sns.set_theme(style='whitegrid')
plt.figure(figsize=(20, 10), dpi=300)
plt.rcParams['font.family'] = 'NanumGothicOTF'

# 사용일자에서 '월' 변수를 추출하여 추가
df['사용일자'] = pd.to_datetime(df['사용일자'])  # 사용일자를 datetime 형식으로 변환
df['월'] = df['사용일자'].dt.to_period('M')  # '월' 변수 생성 (YYYY-MM 형태)

# 2023년까지의 데이터만 필터링
df_filtered = df[df['사용일자'] <= '2023-12-31']  # 2023년까지 데이터 선택

# 금액 추세 수치 계산
monthly_trend = df_filtered.groupby(['월', '용도'])['사용금액'].sum().reset_index()
trend_pivot = monthly_trend.pivot(index='월', columns='용도', values='사용금액').fillna(0)
trend_pivot.index = trend_pivot.index.to_timestamp().strftime('%Y-%m')  # Timestamp → 'YYYY-MM' 형태 변환

# 각 용도별 총 사용 금액 계산
total_usage_by_type = trend_pivot.sum(axis=0)

# 총 사용 금액이 가장 많은 상위 5개의 용도 선택
top_5_usage_types = total_usage_by_type.nlargest(5).index

# 상위 5개의 용도만 필터링
filtered_trend_pivot = trend_pivot[top_5_usage_types]

# Seaborn 라인 차트 생성
for usage_type in filtered_trend_pivot.columns:
    sns.lineplot(x=filtered_trend_pivot.index, y=filtered_trend_pivot[usage_type], marker='o', label=usage_type)

# 제목 및 레이블 설정
plt.title('월별 용도별 사용금액 추세 (Top 5)', fontsize=16, fontweight='bold')
plt.xlabel('월', fontsize=14, fontweight='bold')
plt.ylabel('사용금액', fontsize=14, fontweight='bold')

# y축 금액 포맷을 일반 숫자 형식으로 변경 (콤마 포함)
ax = plt.gca()  # 현재 축 가져오기

# FuncFormatter를 사용하여 금액을 콤마로 구분된 형식으로 변경
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{int(x):,}'))

# x축 레이블 간격을 조정하고 레이블을 45도 회전
plt.xticks(rotation=45, ha='right')  # ha='right'는 레이블을 오른쪽에 정렬

# Legend를 그래프 밖으로 위치 조정
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=14)

# 그리드 추가
plt.grid(True)

# 그래프 저장
line_chart_path = os.path.join(company_result_root, '결과2_용도 및 월별 추세.png')
plt.tight_layout()
plt.savefig(line_chart_path, bbox_inches='tight')
plt.close()

print('용도 및 월별 금액 추세 그래프 (2023년까지) 저장 완료')

용도/월별 금액 추세 계산 중
용도 및 월별 금액 추세 그래프 (2023년까지) 저장 완료


#### **3) 매입(카드/페이) : 비용 적정성 결과**

In [32]:
# 분석 3 : 용도별 전월 대비 비용 적정성 분석 결과
print('전월 대비 비용 적정성 계산 중')

# 금액 추세 수치 계산
monthly_trend = df_filtered.groupby(['월', '용도'])['사용금액'].sum().reset_index()
trend_pivot = monthly_trend.pivot(index='월', columns='용도', values='사용금액').fillna(0)
trend_pivot.index = trend_pivot.index.to_timestamp().strftime('%Y-%m')  # Timestamp → 'YYYY-MM' 형태 변환

# 전월 대비 이번 달의 적정성 평가
comparison_results = pd.DataFrame()

# 첫 달은 비교 없이 그대로 금액을 표시
first_month_result = pd.DataFrame(trend_pivot.iloc[0].apply(lambda x: f'{int(x):,}')).T
first_month_result.index = [trend_pivot.index[0]]
comparison_results = pd.concat([comparison_results, first_month_result], ignore_index=False)

for k in range(1, len(trend_pivot)):  # 첫 달 이후 달부터 평가
    last_month = trend_pivot.iloc[k - 1]  # 전월 데이터
    current_month = trend_pivot.iloc[k]  # 이번 달 데이터
    comparison = current_month - last_month  # 전월과의 차이 계산
    
    # 적정성 평가
    comparison_eval = comparison.apply(
        lambda x: '낮음' if x < 0 else ('높음' if x > 0 else '보통')  # 전월 대비 적정성 평가
    )
    
    # '비용 적정성(변동액)' 형태로 정리
    result_with_values = comparison_eval + '(' + comparison.fillna(0).round(0).astype(int).apply(lambda x: f'{x:,}') + ')'
    
    # 결과 정리
    monthly_result = pd.DataFrame(result_with_values).T
    monthly_result.index = [trend_pivot.index[k]]
    comparison_results = pd.concat([comparison_results, monthly_result], ignore_index=False)

# 엑셀 저장 경로
output_file = os.path.join(company_result_root, '결과3_전월 대비 비용 적정성 결과.xlsx')

# Excel 파일 생성
wb = Workbook()
ws = wb.active
ws.title = "전월 대비 비용 적정성 분석 결과"

# 첫 번째 행에 열 제목을 입력
ws.append(['날짜'] + list(comparison_results.columns))

# 데이터프레임을 엑셀 시트로 전송
for r_idx, row in enumerate(dataframe_to_rows(comparison_results, index=True, header=True), 1):
    for c_idx, value in enumerate(row, 1):
        ws.cell(row=r_idx, column=c_idx, value=value)
        
        # 첫 번째 달(첫 행)에는 색상을 부여하지 않음
        if r_idx > 1 and c_idx > 1 and isinstance(value, str):  # 첫 번째 행과 열은 제외하고 색상 설정
            if '(' in value:  # 괄호가 있는지 확인
                eval_text = value.split('(')[0]  # "낮음", "보통", "높음" 추출
                number_value = value.split('(')[1].replace(')', '')  # 금액 부분만 추출
                
                if eval_text == '낮음':
                    ws.cell(row=r_idx, column=c_idx, value=number_value).fill = PatternFill(start_color="32CD32", end_color="32CD32", fill_type="solid")  # Limegreen
                elif eval_text == '보통':
                    ws.cell(row=r_idx, column=c_idx, value=number_value).fill = PatternFill(start_color="FFD700", end_color="FFD700", fill_type="solid")  # Gold
                elif eval_text == '높음':
                    ws.cell(row=r_idx, column=c_idx, value=number_value).fill = PatternFill(start_color="FF6347", end_color="FF6347", fill_type="solid")  # Tomato

# 엑셀 파일 저장
wb.save(output_file)
print("전월 대비 비용 적정성 결과 저장 완료")

전월 대비 비용 적정성 계산 중
전월 대비 비용 적정성 결과 저장 완료


#### **4) 매입(카드/페이) : 향후 추세 분석/예측 결과**

In [34]:
# 경고 무시 설정
warnings.filterwarnings("ignore", category=FutureWarning)

# Seaborn 스타일 설정 및 글씨 설정
sns.set_theme(style='whitegrid')
plt.figure(figsize=(20, 10), dpi=300)
plt.rcParams['font.family'] = 'NanumGothicOTF'

# 분석 4 : 미래 12개월 금액 추세 예측 결과(LSTM + MAPE 평가)
print('향후 12개월 추세 예측 및 위험도 평가 실행 중')
forecast_results = {} # 추세 예측 결과
risk_eval_results = {} # 위험도 평가 결과
mape_results = {}  # MAPE 평가 결과
forecast_plots = {} # 용도별 예측 그래프 경로

for col in tqdm(trend_pivot.columns, desc='Forecasting'):
    if len(trend_pivot[col].dropna()) < 3:  # 예측 시도 전에 데이터 길이 체크
        print(f'예측 불가 : {col} → 데이터가 3개 미만')
        forecast_results[col] = pd.Series([None] * 12)  # 예측 불가
        risk_eval_results[col] = [None] * 12  # 예측 불가
        mape_results[col] = None  # MAPE 계산 불가
        continue
    
    # 0 값 비율 계산
    zero_ratio = (trend_pivot[col] == 0).mean()
    
    # 0 값 비율이 10%를 넘으면 예측을 건너뜀
    if zero_ratio > 0.1:  # 10% 이상 0이면 예측 건너뛰기
        print(f'{col}: 예측 건너뜀 (0 값 비율 = {zero_ratio:.2%})')
        forecast_results[col] = pd.Series([None] * 12)  # 예측 불가
        risk_eval_results[col] = [None] * 12  # 예측 불가
        mape_results[col] = None  # MAPE 계산 불가
        continue

    try:
        # LSTM 모델 적용
        print('----------------------------------------------')
        print(f'{col}: LSTM Model 사용')
        forecast = lstm_forecast(trend_pivot[col].dropna(), steps=12)
        
        # 예측 결과 저장
        forecast_results[col] = forecast
        
        # MAPE 계산 (마지막 12개월의 실제 값과 비교)
        actual_values = trend_pivot[col].dropna()[-12:]
        if len(actual_values) == 12:
            mape = calculate_mape(actual_values, forecast)
            mape_results[col] = mape
            print(f'{col} MAPE: {mape:.2f}%')
        else:
            print(f'{col} MAPE 계산 불가: 데이터가 충분하지 않음')
            mape_results[col] = None
        
        # 인덱스를 명시적으로 날짜로 변환 (tz 오류 해결)
        trend_pivot.index = pd.to_datetime(trend_pivot.index, format='%Y-%m')
        
        # Seaborn lineplot을 사용하여 실제값과 예측값 그리기
        plt.figure(figsize=(20, 10), dpi=300)  # 모든 그래프의 크기를 동일하게 설정
        sns.lineplot(x=trend_pivot.index, y=trend_pivot[col], label='실제값', marker='o', color='blue')
        sns.lineplot(x=pd.date_range(start=trend_pivot.index[-1], periods=12, freq='M'), y=forecast, label='예측값 (LSTM Model)', linestyle='--', marker='x', color='red')

        # 제목 및 레이블 설정
        plt.title(f'{col} 향후 12개월 예상 금액', fontsize=16, fontweight='bold')
        plt.xlabel('날짜', fontsize=14, fontweight='bold')
        plt.ylabel('금액', fontsize=14, fontweight='bold')

        # y축 금액 포맷을 일반 숫자 형식으로 변경 (콤마 포함)
        ax = plt.gca()  # 현재 축 가져오기

        # FuncFormatter를 사용하여 금액을 콤마로 구분된 형식으로 변경
        ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{int(x):,}'))
        
        # y축의 값을 0부터 시작하도록 설정
        plt.ylim(bottom=0)

        # x축 레이블 간격을 조정하고 레이블을 45도 회전
        plt.xticks(rotation=45, ha='right')  # ha='right'는 레이블을 오른쪽에 정렬

        # Legend를 그래프 밖으로 위치 조정
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=14)

        # 그래프 저장
        forecast_plot_path = os.path.join(company_result_root, f'결과4_용도 {col} 예측 그래프.png')
        plt.tight_layout()
        plt.savefig(forecast_plot_path)
        plt.close()
        
        # 그래프 경로 저장
        forecast_plots[col] = forecast_plot_path
        
    except Exception as e:
        # 예측 실패 시 None 값 처리
        print(f'예측 오류 {col}: {e}')
        forecast_results[col] = pd.Series([None] * 12)
        risk_eval_results[col] = [None] * 12
        mape_results[col] = None
    
    print('----------------------------------------------')

# 예측 결과 + 위험도 평가 + MAPE 결과 정리
forecast_df = pd.DataFrame(forecast_results)
start_date = df['사용일자'].max()
forecast_df.index = pd.date_range(start=start_date, periods=12, freq='M')
forecast_df.index = forecast_df.index.strftime('%Y-%m')  # Index 'yyyy-mm' 형태 변환

# MAPE 결과 추가
mape_df = pd.DataFrame(mape_results, index=[0])
mape_df.index = ['MAPE']

# 예측 결과 + MAPE 평가 취합 및 정렬
forecast_combined_df = pd.concat([forecast_df, mape_df], axis=0)

# 엑셀 저장 경로 설정
output_file = os.path.join(company_result_root, '결과4_추세 예측 결과.xlsx')

# Excel 파일 생성 및 저장
wb = Workbook()
ws = wb.active
ws.title = "예측 결과"

# 첫 번째 행에 열 제목을 입력
ws.append(['날짜'] + list(forecast_combined_df.columns))

# 데이터프레임을 엑셀 시트로 전송 및 색상 적용
for r_idx, row in enumerate(forecast_combined_df.itertuples(), 1):
    for c_idx, value in enumerate(row, 0):  # 0에서 시작하여 열을 일치시킴
        if c_idx == 0:
            # 첫 번째 열에 날짜를 입력
            ws.cell(row=r_idx + 1, column=1, value=forecast_combined_df.index[r_idx - 1])
        else:
            # 나머지 열에 값 입력
            ws.cell(row=r_idx + 1, column=c_idx + 1, value=value)

            # MAPE 행에는 색상을 적용하지 않음
            if forecast_combined_df.index[r_idx - 1] != 'MAPE' and pd.notna(value):
                # 과거 평균값과 비교하여 색상 적용
                col_name = forecast_combined_df.columns[c_idx - 1]  # c_idx-1로 수정하여 열을 올바르게 참조
                
                # 예측하기 직전 달까지의 평균값 계산 (MAPE와 현재 행을 제외한 값들로 평균 계산)
                avg_past = trend_pivot[col_name].mean()

                # 값이 평균보다 작으면 초록색, 크면 빨간색, 비슷하면 주황색
                if value < avg_past:
                    ws.cell(row=r_idx + 1, column=c_idx + 1).fill = PatternFill(start_color="32CD32", end_color="32CD32", fill_type="solid")  # Limegreen
                elif value > avg_past:
                    ws.cell(row=r_idx + 1, column=c_idx + 1).fill = PatternFill(start_color="FF6347", end_color="FF6347", fill_type="solid")  # Tomato
                else:
                    ws.cell(row=r_idx + 1, column=c_idx + 1).fill = PatternFill(start_color="FFD700", end_color="FFD700", fill_type="solid")  # Gold

# 엑셀 파일 저장
wb.save(output_file)
print("예측 결과 및 위험도 분석 엑셀 저장 완료")

향후 12개월 추세 예측 및 위험도 평가 실행 중


Forecasting:   0%|          | 0/15 [00:00<?, ?it/s]

가지급금 출금: 예측 건너뜀 (0 값 비율 = 94.44%)
기타비용: 예측 건너뜀 (0 값 비율 = 63.89%)
도서인쇄비: 예측 건너뜀 (0 값 비율 = 22.22%)
보험료: 예측 건너뜀 (0 값 비율 = 77.78%)
----------------------------------------------
복리후생비: LSTM Model 사용
복리후생비 MAPE: 21.29%


Forecasting:  33%|███▎      | 5/15 [00:08<00:16,  1.61s/it]

----------------------------------------------
----------------------------------------------
소모품비: LSTM Model 사용
소모품비 MAPE: 27.00%


...

Forecasting:  73%|███████▎  | 11/15 [00:22<00:08,  2.04s/it]

----------------------------------------------
----------------------------------------------
차량유지비: LSTM Model 사용
차량유지비 MAPE: 53.29%


Forecasting: 100%|██████████| 15/15 [00:30<00:00,  2.02s/it]

----------------------------------------------
출장비: 예측 건너뜀 (0 값 비율 = 91.67%)
통신비: 예측 건너뜀 (0 값 비율 = 80.56%)
회식비: 예측 건너뜀 (0 값 비율 = 97.22%)
예측 결과 및 위험도 분석 엑셀 저장 완료


<Figure size 6000x3000 with 0 Axes>

---

#### **3. 매입(현금영수증) Data Load**

In [37]:
# '매입(현금영수증)' 파일명 검색
file_name = [file for file in os.listdir() if '매입(현금영수증)' in file]  # 매입(현금영수증) 파일 1개

# 파일 존재 여부 확인
if len(file_name) > 0:
    # Data Frame
    df = pd.read_excel(file_name[0], skiprows=1)
    # Data Frame Head
    df_without_account = df.drop(columns=['발급수단', '내용'])
    print(df_without_account.head())
else:
    # 파일 없음 오류 문구
    print("매입(현금영수증) 파일이 없습니다.")

               사용일시  증빙    사용금액   용도  프로젝트/현장             비고  메모
0  2022-01-07 13:24  현영  440000  임차료      NaN  2022-01-10 계좌 NaN
1  2021-12-07 12:14  현영  440000  임차료      NaN  2021-12-10 계좌 NaN
2  2021-11-09 22:44  현영  440000  임차료      NaN  2021-11-10 계좌 NaN
3  2021-10-07 17:06  현영  440000  임차료      NaN  2021-10-08 계좌 NaN
4  2021-09-07 23:12  현영  440000  임차료      NaN  2021-09-10 계좌 NaN


In [38]:
# 회사명에 따른 결과물 저장 경로
result_root = os.path.join(root, 'Model(2nd)')
company_result_root = os.path.join(result_root, company_name, '3_매입(현금영수증)')
if not os.path.isdir(company_result_root):
    os.makedirs(company_result_root)

#### **1) 매입(현금영수증) : 비율 수치 & Graph**

In [39]:
# 1) 용도별 비율 수치 계산
print('용도별 비율 계산 중')
usage_summary = df.groupby('용도')['사용금액'].sum().reset_index()  # '사용금액'
usage_summary['비율'] = round((usage_summary['사용금액'] / usage_summary['사용금액'].sum()) * 100, 3)

# '사용금액' 열에 콤마를 추가한 형식으로 변환
usage_summary['사용금액'] = usage_summary['사용금액'].apply(lambda x: f'{int(x):,}')

print('----------------------------------------------')
print(usage_summary)
print('----------------------------------------------')

# 결과물 최종 경로
output_file = os.path.join(company_result_root, '결과1_용도 비율 표.xlsx')

# 엑셀 파일로 저장
usage_summary.to_excel(output_file, index=False)
print('용도별 비율 표 Excel 저장 완료')

용도별 비율 계산 중
----------------------------------------------
    용도       사용금액     비율
0  임차료  5,720,000  100.0
----------------------------------------------
용도별 비율 표 Excel 저장 완료


In [40]:
# 2) 3% 이상인 항목만 그래프에 표시 (표에는 모두 포함)
filtered_data = usage_summary[usage_summary['비율'] >= 3]

# Seaborn 스타일 설정 및 글씨 설정
sns.set_theme(style='whitegrid')
plt.figure(figsize=(10, 6), dpi=300)
plt.rcParams['font.family'] = 'NanumGothicOTF' # 'NanumGothic' or 'NanumGothicOTF'

# 컬러맵 설정
cmap = plt.get_cmap('tab20')
colors = [cmap(i / len(filtered_data)) for i in range(len(filtered_data))]

# Seaborn을 사용한 barplot 생성
bar_plot = sns.barplot(x='비율', y='용도', data=filtered_data,
                       hue='용도', palette='tab20', dodge=False, legend=False)

# 제목 및 레이블 설정에 폰트 적용
bar_plot.set_title('용도별 사용금액 비율 (3% 이상만 표시)', fontsize=16, fontweight='bold')
bar_plot.set_xlabel('비율 (%)', fontsize=14, fontweight='bold')
bar_plot.set_ylabel('용도', fontsize=14, fontweight='bold')

# 막대마다 비율(%) 수치 표시
for index, value in enumerate(filtered_data['비율']):
    plt.text(value + 0.5, index, f'{value:.1f}%', va='center', fontsize=12, color='black')  # 수치를 막대 옆에 표시

# 그래프 저장
graph_path = os.path.join(company_result_root, '결과1_용도 비율 그래프.png')
plt.tight_layout()
plt.savefig(graph_path)
plt.close()

print('용도별 비율 그래프 저장 완료')

용도별 비율 그래프 저장 완료


#### **2) 매입(현금영수증) : 월별 추세 수치 & Graph**

In [41]:
# 분석 2 : 용도/월(Month)별 금액 추세 수치 + Graph
print('용도/월별 금액 추세 계산 중')

# Seaborn 스타일 설정 및 글씨 설정
sns.set_theme(style='whitegrid')
plt.figure(figsize=(20, 10), dpi=300)
plt.rcParams['font.family'] = 'NanumGothicOTF'

# 사용일시에서 '월' 변수를 추출하여 추가
df['사용일시'] = pd.to_datetime(df['사용일시'])  # 사용일시를 datetime 형식으로 변환
df['월'] = df['사용일시'].dt.to_period('M')  # '월' 변수 생성 (YYYY-MM 형태)

# 2023년까지의 데이터만 필터링
df_filtered = df[df['사용일시'] <= '2023-12-31']  # 2023년까지 데이터 선택

# 금액 추세 수치 계산
monthly_trend = df_filtered.groupby(['월', '용도'])['사용금액'].sum().reset_index()
trend_pivot = monthly_trend.pivot(index='월', columns='용도', values='사용금액').fillna(0)
trend_pivot.index = trend_pivot.index.to_timestamp().strftime('%Y-%m')  # Timestamp → 'YYYY-MM' 형태 변환

# 각 용도별 총 사용 금액 계산
total_usage_by_type = trend_pivot.sum(axis=0)

# 총 사용 금액이 가장 많은 상위 5개의 용도 선택
top_5_usage_types = total_usage_by_type.nlargest(5).index

# 상위 5개의 용도만 필터링
filtered_trend_pivot = trend_pivot[top_5_usage_types]

# Seaborn 라인 차트 생성
for usage_type in filtered_trend_pivot.columns:
    sns.lineplot(x=filtered_trend_pivot.index, y=filtered_trend_pivot[usage_type], marker='o', label=usage_type)

# 제목 및 레이블 설정
plt.title('월별 용도별 사용금액 추세 (Top 5)', fontsize=16, fontweight='bold')
plt.xlabel('월', fontsize=14, fontweight='bold')
plt.ylabel('사용금액', fontsize=14, fontweight='bold')

# y축 금액 포맷을 일반 숫자 형식으로 변경 (콤마 포함)
ax = plt.gca()  # 현재 축 가져오기

# FuncFormatter를 사용하여 금액을 콤마로 구분된 형식으로 변경
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{int(x):,}'))

# x축 레이블 간격을 조정하고 레이블을 45도 회전
plt.xticks(rotation=45, ha='right')  # ha='right'는 레이블을 오른쪽에 정렬

# Legend를 그래프 밖으로 위치 조정
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=14)

# 그리드 추가
plt.grid(True)

# 그래프 저장
line_chart_path = os.path.join(company_result_root, '결과2_용도 및 월별 추세.png')
plt.tight_layout()
plt.savefig(line_chart_path, bbox_inches='tight')
plt.close()

print('용도 및 월별 금액 추세 그래프 (2023년까지) 저장 완료')

용도/월별 금액 추세 계산 중
용도 및 월별 금액 추세 그래프 (2023년까지) 저장 완료


#### **3) 매입(현금영수증) : 비용 적정성 결과**

In [42]:
# 분석 3 : 용도별 전월 대비 비용 적정성 분석 결과
print('전월 대비 비용 적정성 계산 중')

# 금액 추세 수치 계산
monthly_trend = df_filtered.groupby(['월', '용도'])['사용금액'].sum().reset_index()
trend_pivot = monthly_trend.pivot(index='월', columns='용도', values='사용금액').fillna(0)
trend_pivot.index = trend_pivot.index.to_timestamp().strftime('%Y-%m')  # Timestamp → 'YYYY-MM' 형태 변환

# 전월 대비 이번 달의 적정성 평가
comparison_results = pd.DataFrame()

# 첫 달은 비교 없이 그대로 금액을 표시
first_month_result = pd.DataFrame(trend_pivot.iloc[0].apply(lambda x: f'{int(x):,}')).T
first_month_result.index = [trend_pivot.index[0]]
comparison_results = pd.concat([comparison_results, first_month_result], ignore_index=False)

for k in range(1, len(trend_pivot)):  # 첫 달 이후 달부터 평가
    last_month = trend_pivot.iloc[k - 1]  # 전월 데이터
    current_month = trend_pivot.iloc[k]  # 이번 달 데이터
    comparison = current_month - last_month  # 전월과의 차이 계산
    
    # 적정성 평가
    comparison_eval = comparison.apply(
        lambda x: '낮음' if x < 0 else ('높음' if x > 0 else '보통')  # 전월 대비 적정성 평가
    )
    
    # '비용 적정성(변동액)' 형태로 정리
    result_with_values = comparison_eval + '(' + comparison.fillna(0).round(0).astype(int).apply(lambda x: f'{x:,}') + ')'
    
    # 결과 정리
    monthly_result = pd.DataFrame(result_with_values).T
    monthly_result.index = [trend_pivot.index[k]]
    comparison_results = pd.concat([comparison_results, monthly_result], ignore_index=False)

# 엑셀 저장 경로
output_file = os.path.join(company_result_root, '결과3_전월 대비 비용 적정성 결과.xlsx')

# Excel 파일 생성
wb = Workbook()
ws = wb.active
ws.title = "전월 대비 비용 적정성 분석 결과"

# 첫 번째 행에 열 제목을 입력
ws.append(['날짜'] + list(comparison_results.columns))

# 데이터프레임을 엑셀 시트로 전송
for r_idx, row in enumerate(dataframe_to_rows(comparison_results, index=True, header=True), 1):
    for c_idx, value in enumerate(row, 1):
        ws.cell(row=r_idx, column=c_idx, value=value)
        
        # 첫 번째 달(첫 행)에는 색상을 부여하지 않음
        if r_idx > 1 and c_idx > 1 and isinstance(value, str):  # 첫 번째 행과 열은 제외하고 색상 설정
            if '(' in value:  # 괄호가 있는지 확인
                eval_text = value.split('(')[0]  # "낮음", "보통", "높음" 추출
                number_value = value.split('(')[1].replace(')', '')  # 금액 부분만 추출
                
                if eval_text == '낮음':
                    ws.cell(row=r_idx, column=c_idx, value=number_value).fill = PatternFill(start_color="32CD32", end_color="32CD32", fill_type="solid")  # Limegreen
                elif eval_text == '보통':
                    ws.cell(row=r_idx, column=c_idx, value=number_value).fill = PatternFill(start_color="FFD700", end_color="FFD700", fill_type="solid")  # Gold
                elif eval_text == '높음':
                    ws.cell(row=r_idx, column=c_idx, value=number_value).fill = PatternFill(start_color="FF6347", end_color="FF6347", fill_type="solid")  # Tomato

# 엑셀 파일 저장
wb.save(output_file)
print("전월 대비 비용 적정성 결과 저장 완료")

전월 대비 비용 적정성 계산 중
전월 대비 비용 적정성 결과 저장 완료


#### **4) 매입(현금영수증) : 향후 추세 분석/예측 결과**

In [43]:
# 경고 무시 설정
warnings.filterwarnings("ignore", category=FutureWarning)

# Seaborn 스타일 설정 및 글씨 설정
sns.set_theme(style='whitegrid')
plt.figure(figsize=(20, 10), dpi=300)
plt.rcParams['font.family'] = 'NanumGothicOTF'

# 분석 4 : 미래 12개월 금액 추세 예측 결과(LSTM + MAPE 평가)
print('향후 12개월 추세 예측 및 위험도 평가 실행 중')
forecast_results = {} # 추세 예측 결과
risk_eval_results = {} # 위험도 평가 결과
mape_results = {}  # MAPE 평가 결과
forecast_plots = {} # 용도별 예측 그래프 경로

for col in tqdm(trend_pivot.columns, desc='Forecasting'):
    if len(trend_pivot[col].dropna()) < 3:  # 예측 시도 전에 데이터 길이 체크
        print(f'예측 불가 : {col} → 데이터가 3개 미만')
        forecast_results[col] = pd.Series([None] * 12)  # 예측 불가
        risk_eval_results[col] = [None] * 12  # 예측 불가
        mape_results[col] = None  # MAPE 계산 불가
        continue
    
    # 0 값 비율 계산
    zero_ratio = (trend_pivot[col] == 0).mean()
    
    # 0 값 비율이 10%를 넘으면 예측을 건너뜀
    if zero_ratio > 0.1:  # 10% 이상 0이면 예측 건너뛰기
        print(f'{col}: 예측 건너뜀 (0 값 비율 = {zero_ratio:.2%})')
        forecast_results[col] = pd.Series([None] * 12)  # 예측 불가
        risk_eval_results[col] = [None] * 12  # 예측 불가
        mape_results[col] = None  # MAPE 계산 불가
        continue

    try:
        # LSTM 모델 적용
        print('----------------------------------------------')
        print(f'{col}: LSTM Model 사용')
        forecast = lstm_forecast(trend_pivot[col].dropna(), steps=12)
        
        # 예측 결과 저장
        forecast_results[col] = forecast
        
        # MAPE 계산 (마지막 12개월의 실제 값과 비교)
        actual_values = trend_pivot[col].dropna()[-12:]
        if len(actual_values) == 12:
            mape = calculate_mape(actual_values, forecast)
            mape_results[col] = mape
            print(f'{col} MAPE: {mape:.2f}%')
        else:
            print(f'{col} MAPE 계산 불가: 데이터가 충분하지 않음')
            mape_results[col] = None
        
        # 인덱스를 명시적으로 날짜로 변환 (tz 오류 해결)
        trend_pivot.index = pd.to_datetime(trend_pivot.index, format='%Y-%m')
        
        # Seaborn lineplot을 사용하여 실제값과 예측값 그리기
        plt.figure(figsize=(20, 10), dpi=300)  # 모든 그래프의 크기를 동일하게 설정
        sns.lineplot(x=trend_pivot.index, y=trend_pivot[col], label='실제값', marker='o', color='blue')
        sns.lineplot(x=pd.date_range(start=trend_pivot.index[-1], periods=12, freq='M'), y=forecast, label='예측값 (LSTM Model)', linestyle='--', marker='x', color='red')

        # 제목 및 레이블 설정
        plt.title(f'{col} 향후 12개월 예상 금액', fontsize=16, fontweight='bold')
        plt.xlabel('날짜', fontsize=14, fontweight='bold')
        plt.ylabel('금액', fontsize=14, fontweight='bold')

        # y축 금액 포맷을 일반 숫자 형식으로 변경 (콤마 포함)
        ax = plt.gca()  # 현재 축 가져오기

        # FuncFormatter를 사용하여 금액을 콤마로 구분된 형식으로 변경
        ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{int(x):,}'))
        
        # y축의 값을 0부터 시작하도록 설정
        plt.ylim(bottom=0)

        # x축 레이블 간격을 조정하고 레이블을 45도 회전
        plt.xticks(rotation=45, ha='right')  # ha='right'는 레이블을 오른쪽에 정렬

        # Legend를 그래프 밖으로 위치 조정
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=14)

        # 그래프 저장
        forecast_plot_path = os.path.join(company_result_root, f'결과4_용도 {col} 예측 그래프.png')
        plt.tight_layout()
        plt.savefig(forecast_plot_path)
        plt.close()
        
        # 그래프 경로 저장
        forecast_plots[col] = forecast_plot_path
        
    except Exception as e:
        # 예측 실패 시 None 값 처리
        print(f'예측 오류 {col}: {e}')
        forecast_results[col] = pd.Series([None] * 12)
        risk_eval_results[col] = [None] * 12
        mape_results[col] = None
    
    print('----------------------------------------------')

# 예측 결과 + 위험도 평가 + MAPE 결과 정리
forecast_df = pd.DataFrame(forecast_results)
start_date = df['사용일시'].max()
forecast_df.index = pd.date_range(start=start_date, periods=12, freq='M')
forecast_df.index = forecast_df.index.strftime('%Y-%m')  # Index 'yyyy-mm' 형태 변환

# MAPE 결과 추가
mape_df = pd.DataFrame(mape_results, index=[0])
mape_df.index = ['MAPE']

# 예측 결과 + MAPE 평가 취합 및 정렬
forecast_combined_df = pd.concat([forecast_df, mape_df], axis=0)

# 엑셀 저장 경로 설정
output_file = os.path.join(company_result_root, '결과4_추세 예측 결과.xlsx')

# Excel 파일 생성 및 저장
wb = Workbook()
ws = wb.active
ws.title = "예측 결과"

# 첫 번째 행에 열 제목을 입력
ws.append(['날짜'] + list(forecast_combined_df.columns))

# 데이터프레임을 엑셀 시트로 전송 및 색상 적용
for r_idx, row in enumerate(forecast_combined_df.itertuples(), 1):
    for c_idx, value in enumerate(row, 0):  # 0에서 시작하여 열을 일치시킴
        if c_idx == 0:
            # 첫 번째 열에 날짜를 입력
            ws.cell(row=r_idx + 1, column=1, value=forecast_combined_df.index[r_idx - 1])
        else:
            # 나머지 열에 값 입력
            ws.cell(row=r_idx + 1, column=c_idx + 1, value=value)

            # MAPE 행에는 색상을 적용하지 않음
            if forecast_combined_df.index[r_idx - 1] != 'MAPE' and pd.notna(value):
                # 과거 평균값과 비교하여 색상 적용
                col_name = forecast_combined_df.columns[c_idx - 1]  # c_idx-1로 수정하여 열을 올바르게 참조
                
                # 예측하기 직전 달까지의 평균값 계산 (MAPE와 현재 행을 제외한 값들로 평균 계산)
                avg_past = trend_pivot[col_name].mean()

                # 값이 평균보다 작으면 초록색, 크면 빨간색, 비슷하면 주황색
                if value < avg_past:
                    ws.cell(row=r_idx + 1, column=c_idx + 1).fill = PatternFill(start_color="32CD32", end_color="32CD32", fill_type="solid")  # Limegreen
                elif value > avg_past:
                    ws.cell(row=r_idx + 1, column=c_idx + 1).fill = PatternFill(start_color="FF6347", end_color="FF6347", fill_type="solid")  # Tomato
                else:
                    ws.cell(row=r_idx + 1, column=c_idx + 1).fill = PatternFill(start_color="FFD700", end_color="FFD700", fill_type="solid")  # Gold

# 엑셀 파일 저장
wb.save(output_file)
print("예측 결과 및 위험도 분석 엑셀 저장 완료")

향후 12개월 추세 예측 및 위험도 평가 실행 중


Forecasting:   0%|          | 0/1 [00:00<?, ?it/s]

----------------------------------------------
임차료: LSTM Model 사용
임차료 MAPE: 0.00%


Forecasting: 100%|██████████| 1/1 [00:08<00:00,  8.44s/it]

----------------------------------------------
예측 결과 및 위험도 분석 엑셀 저장 완료


<Figure size 6000x3000 with 0 Axes>

---

#### **4. 매입(간이영수증) Data Load**

In [44]:
# '매입(간이영수증)' 파일명 검색
file_name = [file for file in os.listdir() if '매입(간이영수증)' in file]  # 매입(간이영수증) 파일 1개

# 파일 존재 여부 확인
if len(file_name) > 0:
    # Data Frame
    df = pd.read_excel(file_name[0], skiprows=1)
    # Data Frame Head(2023)
    df_without_account = df.drop(columns=['사용처', '내용'])
    print(df_without_account[df_without_account['사용일시'].str.contains('2023')].head())
else:
    # 파일 없음 오류 문구
    print("매입(간이영수증) 파일이 없습니다.")

          사용일시  증빙      사용금액  용도  프로젝트/현장  비고   메모
7   2023-12-31  경비  12500000  급여      NaN NaN  NaN
8   2023-11-30  경비  12500000  급여      NaN NaN  NaN
9   2023-10-31  경비  12500000  급여      NaN NaN  NaN
10  2023-09-30  경비  13400000  급여      NaN NaN  NaN
11  2023-08-31  경비  12500000  급여      NaN NaN  NaN


In [45]:
# 회사명에 따른 결과물 저장 경로
result_root = os.path.join(root, 'Model(2nd)')
company_result_root = os.path.join(result_root, company_name, '4_매입(간이영수증)')
if not os.path.isdir(company_result_root):
    os.makedirs(company_result_root)

#### **1) 매입(간이영수증) : 비율 수치 & Graph**

In [46]:
# 1) 용도별 비율 수치 계산
print('용도별 비율 계산 중')
usage_summary = df.groupby('용도')['사용금액'].sum().reset_index()  # '사용금액'
usage_summary['비율'] = round((usage_summary['사용금액'] / usage_summary['사용금액'].sum()) * 100, 3)

# '사용금액' 열에 콤마를 추가한 형식으로 변환
usage_summary['사용금액'] = usage_summary['사용금액'].apply(lambda x: f'{int(x):,}')

print('----------------------------------------------')
print(usage_summary)
print('----------------------------------------------')

# 결과물 최종 경로
output_file = os.path.join(company_result_root, '결과1_용도 비율 표.xlsx')

# 엑셀 파일로 저장
usage_summary.to_excel(output_file, index=False)
print('용도별 비율 표 Excel 저장 완료')

용도별 비율 계산 중
----------------------------------------------
   용도         사용금액     비율
0  급여  404,120,000  100.0
----------------------------------------------
용도별 비율 표 Excel 저장 완료


In [47]:
# 2) 3% 이상인 항목만 그래프에 표시 (표에는 모두 포함)
filtered_data = usage_summary[usage_summary['비율'] >= 3]

# Seaborn 스타일 설정 및 글씨 설정
sns.set_theme(style='whitegrid')
plt.figure(figsize=(10, 6), dpi=300)
plt.rcParams['font.family'] = 'NanumGothicOTF' # 'NanumGothic' or 'NanumGothicOTF'

# 컬러맵 설정
cmap = plt.get_cmap('tab20')
colors = [cmap(i / len(filtered_data)) for i in range(len(filtered_data))]

# Seaborn을 사용한 barplot 생성
bar_plot = sns.barplot(x='비율', y='용도', data=filtered_data,
                       hue='용도', palette='tab20', dodge=False, legend=False)

# 제목 및 레이블 설정에 폰트 적용
bar_plot.set_title('용도별 사용금액 비율 (3% 이상만 표시)', fontsize=16, fontweight='bold')
bar_plot.set_xlabel('비율 (%)', fontsize=14, fontweight='bold')
bar_plot.set_ylabel('용도', fontsize=14, fontweight='bold')

# 막대마다 비율(%) 수치 표시
for index, value in enumerate(filtered_data['비율']):
    plt.text(value + 0.5, index, f'{value:.1f}%', va='center', fontsize=12, color='black')  # 수치를 막대 옆에 표시

# 그래프 저장
graph_path = os.path.join(company_result_root, '결과1_용도 비율 그래프.png')
plt.tight_layout()
plt.savefig(graph_path)
plt.close()

print('용도별 비율 그래프 저장 완료')

용도별 비율 그래프 저장 완료


#### **2) 매입(간이영수증) : 월별 추세 수치 & Graph**

In [48]:
# 분석 2 : 용도/월(Month)별 금액 추세 수치 + Graph
print('용도/월별 금액 추세 계산 중')

# Seaborn 스타일 설정 및 글씨 설정
sns.set_theme(style='whitegrid')
plt.figure(figsize=(20, 10), dpi=300)
plt.rcParams['font.family'] = 'NanumGothicOTF'

# 사용일시에서 '월' 변수를 추출하여 추가
df['사용일시'] = pd.to_datetime(df['사용일시'], errors='coerce')  # 사용일시를 datetime 형식으로 변환
df['월'] = df['사용일시'].dt.to_period('M')  # '월' 변수 생성 (YYYY-MM 형태)

# 2023년까지의 데이터만 필터링
df_filtered = df[df['사용일시'] <= '2023-12-31']  # 2023년까지 데이터 선택

# 금액 추세 수치 계산
monthly_trend = df_filtered.groupby(['월', '용도'])['사용금액'].sum().reset_index()
trend_pivot = monthly_trend.pivot(index='월', columns='용도', values='사용금액').fillna(0)
trend_pivot.index = trend_pivot.index.to_timestamp().strftime('%Y-%m')  # Timestamp → 'YYYY-MM' 형태 변환

# 각 용도별 총 사용 금액 계산
total_usage_by_type = trend_pivot.sum(axis=0)

# 총 사용 금액이 가장 많은 상위 5개의 용도 선택
top_5_usage_types = total_usage_by_type.nlargest(5).index

# 상위 5개의 용도만 필터링
filtered_trend_pivot = trend_pivot[top_5_usage_types]

# Seaborn 라인 차트 생성
for usage_type in filtered_trend_pivot.columns:
    sns.lineplot(x=filtered_trend_pivot.index, y=filtered_trend_pivot[usage_type], marker='o', label=usage_type)

# 제목 및 레이블 설정
plt.title('월별 용도별 사용금액 추세 (Top 5)', fontsize=16, fontweight='bold')
plt.xlabel('월', fontsize=14, fontweight='bold')
plt.ylabel('사용금액', fontsize=14, fontweight='bold')

# y축 금액 포맷을 일반 숫자 형식으로 변경 (콤마 포함)
ax = plt.gca()  # 현재 축 가져오기

# FuncFormatter를 사용하여 금액을 콤마로 구분된 형식으로 변경
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{int(x):,}'))

# x축 레이블 간격을 조정하고 레이블을 45도 회전
plt.xticks(rotation=45, ha='right')  # ha='right'는 레이블을 오른쪽에 정렬

# Legend를 그래프 밖으로 위치 조정
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=14)

# 그리드 추가
plt.grid(True)

# 그래프 저장
line_chart_path = os.path.join(company_result_root, '결과2_용도 및 월별 추세.png')
plt.tight_layout()
plt.savefig(line_chart_path, bbox_inches='tight')
plt.close()

print('용도 및 월별 금액 추세 그래프 (2023년까지) 저장 완료')

용도/월별 금액 추세 계산 중
용도 및 월별 금액 추세 그래프 (2023년까지) 저장 완료


#### **3) 매입(간이영수증) : 비용 적정성 결과**

In [49]:
# 분석 3 : 용도별 전월 대비 비용 적정성 분석 결과
print('전월 대비 비용 적정성 계산 중')

# 금액 추세 수치 계산
monthly_trend = df_filtered.groupby(['월', '용도'])['사용금액'].sum().reset_index()
trend_pivot = monthly_trend.pivot(index='월', columns='용도', values='사용금액').fillna(0)
trend_pivot.index = trend_pivot.index.to_timestamp().strftime('%Y-%m')  # Timestamp → 'YYYY-MM' 형태 변환

# 전월 대비 이번 달의 적정성 평가
comparison_results = pd.DataFrame()

# 첫 달은 비교 없이 그대로 금액을 표시
first_month_result = pd.DataFrame(trend_pivot.iloc[0].apply(lambda x: f'{int(x):,}')).T
first_month_result.index = [trend_pivot.index[0]]
comparison_results = pd.concat([comparison_results, first_month_result], ignore_index=False)

for k in range(1, len(trend_pivot)):  # 첫 달 이후 달부터 평가
    last_month = trend_pivot.iloc[k - 1]  # 전월 데이터
    current_month = trend_pivot.iloc[k]  # 이번 달 데이터
    comparison = current_month - last_month  # 전월과의 차이 계산
    
    # 적정성 평가
    comparison_eval = comparison.apply(
        lambda x: '낮음' if x < 0 else ('높음' if x > 0 else '보통')  # 전월 대비 적정성 평가
    )
    
    # '비용 적정성(변동액)' 형태로 정리
    result_with_values = comparison_eval + '(' + comparison.fillna(0).round(0).astype(int).apply(lambda x: f'{x:,}') + ')'
    
    # 결과 정리
    monthly_result = pd.DataFrame(result_with_values).T
    monthly_result.index = [trend_pivot.index[k]]
    comparison_results = pd.concat([comparison_results, monthly_result], ignore_index=False)

# 엑셀 저장 경로
output_file = os.path.join(company_result_root, '결과3_전월 대비 비용 적정성 결과.xlsx')

# Excel 파일 생성
wb = Workbook()
ws = wb.active
ws.title = "전월 대비 비용 적정성 분석 결과"

# 첫 번째 행에 열 제목을 입력
ws.append(['날짜'] + list(comparison_results.columns))

# 데이터프레임을 엑셀 시트로 전송
for r_idx, row in enumerate(dataframe_to_rows(comparison_results, index=True, header=True), 1):
    for c_idx, value in enumerate(row, 1):
        ws.cell(row=r_idx, column=c_idx, value=value)
        
        # 첫 번째 달(첫 행)에는 색상을 부여하지 않음
        if r_idx > 1 and c_idx > 1 and isinstance(value, str):  # 첫 번째 행과 열은 제외하고 색상 설정
            if '(' in value:  # 괄호가 있는지 확인
                eval_text = value.split('(')[0]  # "낮음", "보통", "높음" 추출
                number_value = value.split('(')[1].replace(')', '')  # 금액 부분만 추출
                
                if eval_text == '낮음':
                    ws.cell(row=r_idx, column=c_idx, value=number_value).fill = PatternFill(start_color="32CD32", end_color="32CD32", fill_type="solid")  # Limegreen
                elif eval_text == '보통':
                    ws.cell(row=r_idx, column=c_idx, value=number_value).fill = PatternFill(start_color="FFD700", end_color="FFD700", fill_type="solid")  # Gold
                elif eval_text == '높음':
                    ws.cell(row=r_idx, column=c_idx, value=number_value).fill = PatternFill(start_color="FF6347", end_color="FF6347", fill_type="solid")  # Tomato

# 엑셀 파일 저장
wb.save(output_file)
print("전월 대비 비용 적정성 결과 저장 완료")

전월 대비 비용 적정성 계산 중
전월 대비 비용 적정성 결과 저장 완료


#### **4) 매입(간이영수증) : 향후 추세 분석/예측 결과**

In [50]:
# 경고 무시 설정
warnings.filterwarnings("ignore", category=FutureWarning)

# Seaborn 스타일 설정 및 글씨 설정
sns.set_theme(style='whitegrid')
plt.figure(figsize=(20, 10), dpi=300)
plt.rcParams['font.family'] = 'NanumGothicOTF'

# 분석 4 : 미래 12개월 금액 추세 예측 결과(LSTM + MAPE 평가)
print('향후 12개월 추세 예측 및 위험도 평가 실행 중')
forecast_results = {} # 추세 예측 결과
risk_eval_results = {} # 위험도 평가 결과
mape_results = {}  # MAPE 평가 결과
forecast_plots = {} # 용도별 예측 그래프 경로

for col in tqdm(trend_pivot.columns, desc='Forecasting'):
    if len(trend_pivot[col].dropna()) < 3:  # 예측 시도 전에 데이터 길이 체크
        print(f'예측 불가 : {col} → 데이터가 3개 미만')
        forecast_results[col] = pd.Series([None] * 12)  # 예측 불가
        risk_eval_results[col] = [None] * 12  # 예측 불가
        mape_results[col] = None  # MAPE 계산 불가
        continue
    
    # 0 값 비율 계산
    zero_ratio = (trend_pivot[col] == 0).mean()
    
    # 0 값 비율이 10%를 넘으면 예측을 건너뜀
    if zero_ratio > 0.1:  # 10% 이상 0이면 예측 건너뛰기
        print(f'{col}: 예측 건너뜀 (0 값 비율 = {zero_ratio:.2%})')
        forecast_results[col] = pd.Series([None] * 12)  # 예측 불가
        risk_eval_results[col] = [None] * 12  # 예측 불가
        mape_results[col] = None  # MAPE 계산 불가
        continue

    try:
        # LSTM 모델 적용
        print('----------------------------------------------')
        print(f'{col}: LSTM Model 사용')
        forecast = lstm_forecast(trend_pivot[col].dropna(), steps=12)
        
        # 예측 결과 저장
        forecast_results[col] = forecast
        
        # MAPE 계산 (마지막 12개월의 실제 값과 비교)
        actual_values = trend_pivot[col].dropna()[-12:]
        if len(actual_values) == 12:
            mape = calculate_mape(actual_values, forecast)
            mape_results[col] = mape
            print(f'{col} MAPE: {mape:.2f}%')
        else:
            print(f'{col} MAPE 계산 불가: 데이터가 충분하지 않음')
            mape_results[col] = None
        
        # 인덱스를 명시적으로 날짜로 변환 (tz 오류 해결)
        trend_pivot.index = pd.to_datetime(trend_pivot.index, format='%Y-%m')
        
        # Seaborn lineplot을 사용하여 실제값과 예측값 그리기
        plt.figure(figsize=(20, 10), dpi=300)  # 모든 그래프의 크기를 동일하게 설정
        sns.lineplot(x=trend_pivot.index, y=trend_pivot[col], label='실제값', marker='o', color='blue')
        sns.lineplot(x=pd.date_range(start=trend_pivot.index[-1], periods=12, freq='M'), y=forecast, label='예측값 (LSTM Model)', linestyle='--', marker='x', color='red')

        # 제목 및 레이블 설정
        plt.title(f'{col} 향후 12개월 예상 금액', fontsize=16, fontweight='bold')
        plt.xlabel('날짜', fontsize=14, fontweight='bold')
        plt.ylabel('금액', fontsize=14, fontweight='bold')

        # y축 금액 포맷을 일반 숫자 형식으로 변경 (콤마 포함)
        ax = plt.gca()  # 현재 축 가져오기

        # FuncFormatter를 사용하여 금액을 콤마로 구분된 형식으로 변경
        ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{int(x):,}'))
        
        # y축의 값을 0부터 시작하도록 설정
        plt.ylim(bottom=0)

        # x축 레이블 간격을 조정하고 레이블을 45도 회전
        plt.xticks(rotation=45, ha='right')  # ha='right'는 레이블을 오른쪽에 정렬

        # Legend를 그래프 밖으로 위치 조정
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=14)

        # 그래프 저장
        forecast_plot_path = os.path.join(company_result_root, f'결과4_용도 {col} 예측 그래프.png')
        plt.tight_layout()
        plt.savefig(forecast_plot_path)
        plt.close()
        
        # 그래프 경로 저장
        forecast_plots[col] = forecast_plot_path
        
    except Exception as e:
        # 예측 실패 시 None 값 처리
        print(f'예측 오류 {col}: {e}')
        forecast_results[col] = pd.Series([None] * 12)
        risk_eval_results[col] = [None] * 12
        mape_results[col] = None
    
    print('----------------------------------------------')

# 예측 결과 + 위험도 평가 + MAPE 결과 정리
forecast_df = pd.DataFrame(forecast_results)
start_date = df['사용일시'].max()
forecast_df.index = pd.date_range(start=start_date, periods=12, freq='M')
forecast_df.index = forecast_df.index.strftime('%Y-%m')  # Index 'yyyy-mm' 형태 변환

# MAPE 결과 추가
mape_df = pd.DataFrame(mape_results, index=[0])
mape_df.index = ['MAPE']

# 예측 결과 + MAPE 평가 취합 및 정렬
forecast_combined_df = pd.concat([forecast_df, mape_df], axis=0)

# 엑셀 저장 경로 설정
output_file = os.path.join(company_result_root, '결과4_추세 예측 결과.xlsx')

# Excel 파일 생성 및 저장
wb = Workbook()
ws = wb.active
ws.title = "예측 결과"

# 첫 번째 행에 열 제목을 입력
ws.append(['날짜'] + list(forecast_combined_df.columns))

# 데이터프레임을 엑셀 시트로 전송 및 색상 적용
for r_idx, row in enumerate(forecast_combined_df.itertuples(), 1):
    for c_idx, value in enumerate(row, 0):  # 0에서 시작하여 열을 일치시킴
        if c_idx == 0:
            # 첫 번째 열에 날짜를 입력
            ws.cell(row=r_idx + 1, column=1, value=forecast_combined_df.index[r_idx - 1])
        else:
            # 나머지 열에 값 입력
            ws.cell(row=r_idx + 1, column=c_idx + 1, value=value)

            # MAPE 행에는 색상을 적용하지 않음
            if forecast_combined_df.index[r_idx - 1] != 'MAPE' and pd.notna(value):
                # 과거 평균값과 비교하여 색상 적용
                col_name = forecast_combined_df.columns[c_idx - 1]  # c_idx-1로 수정하여 열을 올바르게 참조
                
                # 예측하기 직전 달까지의 평균값 계산 (MAPE와 현재 행을 제외한 값들로 평균 계산)
                avg_past = trend_pivot[col_name].mean()

                # 값이 평균보다 작으면 초록색, 크면 빨간색, 비슷하면 주황색
                if value < avg_past:
                    ws.cell(row=r_idx + 1, column=c_idx + 1).fill = PatternFill(start_color="32CD32", end_color="32CD32", fill_type="solid")  # Limegreen
                elif value > avg_past:
                    ws.cell(row=r_idx + 1, column=c_idx + 1).fill = PatternFill(start_color="FF6347", end_color="FF6347", fill_type="solid")  # Tomato
                else:
                    ws.cell(row=r_idx + 1, column=c_idx + 1).fill = PatternFill(start_color="FFD700", end_color="FFD700", fill_type="solid")  # Gold

# 엑셀 파일 저장
wb.save(output_file)
print("예측 결과 및 위험도 분석 엑셀 저장 완료")

향후 12개월 추세 예측 및 위험도 평가 실행 중


Forecasting:   0%|          | 0/1 [00:00<?, ?it/s]

----------------------------------------------
급여: LSTM Model 사용
급여 MAPE: 5.19%


Forecasting: 100%|██████████| 1/1 [00:09<00:00,  9.34s/it]

----------------------------------------------
예측 결과 및 위험도 분석 엑셀 저장 완료


<Figure size 6000x3000 with 0 Axes>